<a href="https://colab.research.google.com/github/duonghung86/Vehicle-trajectory-tracking/blob/master/Codes/VTP_1_04_Summary_of_simple_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple LSTM

Overview on the configuration of all simple models

|Model name|Description| # Car|# vars|# targets|# const_vars| # steps | # futures
|---|:--|:-:|:-:|:-:|:-:|:-:|:-:|
Model 1|Baseline model with multi input time steps|1|1|1|0|**4**|1| 

## Import packages

In [1]:
# General
import numpy as np
import time
# Loading data
import pandas as pd
from io import StringIO, BytesIO
from zipfile import ZipFile
import urllib.request
import os

#import pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

#Evaluation
from sklearn.metrics import mean_squared_error
from math import sqrt

## Load dataset

In [2]:
def url2pd(link):
    with ZipFile(link) as my_zip_file:
        for contained_file in my_zip_file.namelist():
            fzip=my_zip_file.open(contained_file)
            data=fzip.read()
    s=str(data,'utf-8')
    data = StringIO(s) 
    print('Done loading a dataset!')
    return pd.read_csv(data)

In [3]:
filenames = os.listdir('./Data')
url_1 = './Data/' + filenames[0]
df = url2pd(url_1)
df.info()
df.head(3)

Done loading a dataset!
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 18 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   Vehicle_ID    1048575 non-null  int64  
 1   Frame_ID      1048575 non-null  int64  
 2   Total_Frames  1048575 non-null  int64  
 3   Global_Time   1048575 non-null  int64  
 4   Local_X       1048575 non-null  float64
 5   Local_Y       1048575 non-null  float64
 6   Global_X      1048575 non-null  float64
 7   Global_Y      1048575 non-null  float64
 8   v_Length      1048575 non-null  float64
 9   v_Width       1048575 non-null  float64
 10  v_Class       1048575 non-null  int64  
 11  v_Vel         1048575 non-null  float64
 12  v_Acc         1048575 non-null  float64
 13  Lane_ID       1048575 non-null  int64  
 14  Preceeding    1048575 non-null  int64  
 15  Following     1048575 non-null  int64  
 16  Space_Hdwy    1048575 non-null  float64
 17  Tim

,Vehicle_ID,Frame_ID,Total_Frames,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_Length,v_Width,v_Class,v_Vel,v_Acc,Lane_ID,Preceeding,Following,Space_Hdwy,Time_Hdwy
0,2,13,437,1118846980200,16.467196,35.380427,6451137.641,1873344.962,14.5,4.9,2,40.000000,0.000000,2,0,0,0.0,0.0
1,2,14,437,1118846980300,16.446594,39.381608,6451140.329,1873342.000,14.5,4.9,2,40.012349,0.123485,2,0,0,0.0,0.0
2,2,15,437,1118846980400,16.425991,43.381541,6451143.018,1873339.038,14.5,4.9,2,39.999855,-0.124939,2,0,0,0.0,0.0


Let's take a glance at the data. Here are the first few rows:

Next look at the statistics of the dataset:

In [4]:
df.describe().transpose().round(3)

,count,mean,std,min,25%,50%,75%,max
Vehicle_ID,1048575.0,1.533080e+03,790.271,2.000000e+00,9.320000e+02,1.574000e+03,2.210000e+03,2.783000e+03
Frame_ID,1048575.0,4.518249e+03,2412.479,8.000000e+00,2.455000e+03,4.586000e+03,6.598000e+03,8.906000e+03
Total_Frames,1048575.0,5.608770e+02,146.577,1.770000e+02,4.640000e+02,5.180000e+02,6.400000e+02,1.010000e+03
Global_Time,1048575.0,1.118847e+12,241247.914,1.118847e+12,1.118847e+12,1.118847e+12,1.118848e+12,1.118848e+12
Local_X,1048575.0,2.940600e+01,16.666,5.340000e-01,1.728400e+01,2.955700e+01,4.187500e+01,7.347800e+01
Local_Y,1048575.0,1.002056e+03,596.357,1.796600e+01,4.883960e+02,9.640280e+02,1.491548e+03,2.195470e+03
Global_X,1048575.0,6.451838e+06,446.275,6.451107e+06,6.451450e+06,6.451808e+06,6.452205e+06,6.452734e+06
Global_Y,1048575.0,1.872677e+06,397.006,1.871875e+06,1.872352e+06,1.872699e+06,1.873015e+06,1.873365e+06
v_Length,1048575.0,1.463500e+01,4.870,4.000000e+00,1.200000e+01,1.450000e+01,1.650000e+01,7.610000e+01
v_Width,1048575.0,6.132000e+00,1.037,2.000000e+00,5.400000e+00,6.000000e+00,6.900000e+00,8.500000e+00


In [5]:
# Filter time step
print(df.shape)
df = df.iloc[::2,:].copy()
print('After filtering:', df.shape)
#  keep only columns that are useful for now
kept_cols = ['Vehicle_ID', 'Frame_ID', 'Total_Frames', 'Local_X','Local_Y','v_Length', 'v_Width', 'v_Class',
       'v_Vel', 'v_Acc', 'Lane_ID']
df = df[kept_cols]
df.head(3)

(1048575, 18)
After filtering: (524288, 18)


,Vehicle_ID,Frame_ID,Total_Frames,Local_X,Local_Y,v_Length,v_Width,v_Class,v_Vel,v_Acc,Lane_ID
0,2,13,437,16.467196,35.380427,14.5,4.9,2,40.000000,0.000000,2
2,2,15,437,16.425991,43.381541,14.5,4.9,2,39.999855,-0.124939,2
4,2,17,437,16.384804,51.379881,14.5,4.9,2,39.991544,-0.013759,2


In [6]:
'the number of vehicles is {}'.format(len(df.Vehicle_ID.unique()))

'the number of vehicles is 1993'

In [7]:
df.reset_index(drop=True, inplace=True)

# Model 1

|Model name|Description| # Car|# vars|# targets|# const_vars| # steps | # futures
|---|:--|:-:|:-:|:-:|:-:|:-:|:-:|
Model 1|Baseline model with multi input time steps|1|1|1|0|**4**|1| 

## Data preparation

In [8]:
# Let pick the vehicle that had longest historical data
max_frames = df.Total_Frames.max()
print(max_frames)
car_id = df[df.Total_Frames==max_frames].Vehicle_ID.unique()
"ID of the car that have longest data is {} ".format(car_id[0])

1010


'ID of the car that have longest data is 2582 '

In [9]:
simple_df = df[df.Vehicle_ID==car_id[0]].copy()
simple_df.head()

,Vehicle_ID,Frame_ID,Total_Frames,Local_X,Local_Y,v_Length,v_Width,v_Class,v_Vel,v_Acc,Lane_ID
466051,2582,7263,1010,4.725434,36.935259,18.5,6.9,2,45.030000,0.000000,1
466052,2582,7265,1010,4.668727,45.875960,18.5,6.9,2,44.918464,4.280980,1
466053,2582,7267,1010,4.627103,54.924527,18.5,6.9,2,45.298705,1.107846,1
466054,2582,7269,1010,4.591890,63.945685,18.5,6.9,2,44.961476,-2.893200,1
466055,2582,7271,1010,4.543895,72.797674,18.5,6.9,2,44.002607,-5.160000,1


In [10]:
simple_df.shape

(505, 11)

In [11]:
test_data_size = int(simple_df.shape[0]*0.2)
print('test size is',test_data_size)
train_data = simple_df.Local_X[:-test_data_size]
test_data = simple_df.Local_X[-test_data_size:]
print(len(train_data))
print(len(test_data))

test size is 101
404
101


In [12]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1, 1))
train_data_normalized = scaler.fit_transform(train_data.values.reshape(-1, 1))

In [13]:
train_data_normalized = torch.FloatTensor(train_data_normalized).view(-1)

In [14]:
test_data_normalized = scaler.fit_transform(test_data.values.reshape(-1, 1))
test_data_normalized = torch.FloatTensor(test_data_normalized).view(-1)

In [15]:
def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = input_data[i:i+tw]
        train_label = input_data[i+tw:i+tw+1]
        inout_seq.append((train_seq ,train_label))
    return inout_seq

In [16]:
train_window = 12
train_inout_seq = create_inout_sequences(train_data_normalized, train_window)

In [17]:
test_inout_seq = create_inout_sequences(test_data_normalized, train_window)

In [18]:
train_inout_seq[:2]

[(tensor([-0.9709, -0.9777, -0.9827, -0.9870, -0.9927, -0.9983, -1.0000, -0.9956,
          -0.9848, -0.9736, -0.9617, -0.9500]),
  tensor([-0.9383])),
 (tensor([-0.9777, -0.9827, -0.9870, -0.9927, -0.9983, -1.0000, -0.9956, -0.9848,
          -0.9736, -0.9617, -0.9500, -0.9383]),
  tensor([-0.9266]))]

In [19]:
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

In [20]:
model = LSTM()
loss_function = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
print(model)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

LSTM(
  (lstm): LSTM(1, 100)
  (linear): Linear(in_features=100, out_features=1, bias=True)
)


LSTM(
  (lstm): LSTM(1, 100)
  (linear): Linear(in_features=100, out_features=1, bias=True)
)

In [21]:
%time
epochs = 50

for i in range(epochs):
    for seq, labels in train_inout_seq:
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size).requires_grad_().to(device),
                        torch.zeros(1, 1, model.hidden_layer_size).requires_grad_().to(device))
        seq = seq.to(device)
        labels = labels.to(device)
        y_pred = model(seq)

        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()

    if i%10 == 1:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

Wall time: 0 ns
epoch:   1 loss: 0.00022390
epoch:  11 loss: 0.00002021
epoch:  21 loss: 0.00033038
epoch:  31 loss: 0.00034768
epoch:  41 loss: 0.00012526
epoch:  49 loss: 0.0000051398


In [46]:
model.eval()
fut_pred = 12
test_inputs = test_data_normalized.tolist()
for i in range(len(test_inputs)):
    seq = torch.FloatTensor(test_inputs)
    with torch.no_grad():
        model.hidden = (torch.zeros(1, 1, model.hidden_layer_size),
                        torch.zeros(1, 1, model.hidden_layer_size))
        test_inputs.append(model(seq).item())

In [49]:
len(test_inputs)

113

In [34]:
actual_predictions = scaler.inverse_transform(np.array(test_inputs[train_window:] ).reshape(-1, 1))
print(actual_predictions)

[[20.6484481 ]
 [20.77859266]
 [20.98960473]
 [21.29998619]
 [21.7410973 ]
 [22.34563493]
 [23.00569183]
 [23.43833588]
 [23.54771831]
 [23.1987619 ]
 [22.35805961]
 [21.1604181 ]]


In [45]:
rmse = sqrt(mean_squared_error(train_data[-train_window:], actual_predictions))
rmse

2.3530971170679034

### series2seq1: Function that return sequence input and output| Version 1

**Arguments**:

- data: Sequence of observations as a list or NumPy array.
- n_in: Number of lag observations as input (X).
- n_out: Number of observations as output (y).
- dropnan: Boolean whether or not to drop rows with NaN values.
    
**Returns**:
- X: Feature Pandas DataFrame
- y: Label Pandas dataframe

  

In [10]:
def series2seq1(data, n_in=1, n_out=1, dropnan=True):

    dat = data.copy()

    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(dat.shift(i))
        names += ['{}(t-{})'.format(dat.name, i) ]

    # forecast sequence (t, t+1, ... t+n) for selected labels
    for i in range(0, n_out):
        cols.append(dat.shift(-i))
        if i == 0:
            names += ['{}(t)'.format(dat.name)]
        else:
            names += ['{}(t+{})'.format(dat.name, i)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    X = agg.iloc[:,:n_in]
    y = agg.iloc[:,n_in:].copy()
    return X, y
  
# Test the function
X, y = series2seq1(df.Local_X, n_in=2, n_out=1, dropnan=False)
print(X.head())
print(y.head())

   Local_X(t-2)  Local_X(t-1)
0           NaN           NaN
1           NaN     16.467196
2     16.467196     16.425991
3     16.425991     16.384804
4     16.384804     16.342611
   Local_X(t)
0   16.467196
1   16.425991
2   16.384804
3   16.342611
4   16.304035


In [11]:
n_steps = 4
n_labels = 1
n_future = 1
n_features = 1

In [12]:
X, y = series2seq1(simple_df.Local_X, n_in=n_steps, n_out=n_future, dropnan=True)
print(X.head(), X.shape)
print(y.head(), y.shape)

        Local_X(t-4)  Local_X(t-3)  Local_X(t-2)  Local_X(t-1)
466055      4.725434      4.668727      4.627103      4.591890
466056      4.668727      4.627103      4.591890      4.543895
466057      4.627103      4.591890      4.543895      4.497583
466058      4.591890      4.543895      4.497583      4.483431
466059      4.543895      4.497583      4.483431      4.519953 (501, 4)
        Local_X(t)
466055    4.543895
466056    4.497583
466057    4.483431
466058    4.519953
466059    4.609469 (501, 1)


### Split the data set


In [13]:
### Split the data set
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X,y, 
                                                    test_size=0.3, random_state=42)
print(X_train.shape,X_test.shape, y_train.shape, y_test.shape)

(350, 4) (151, 4) (350, 1) (151, 1)


In [15]:
### Standardize the data
train_mean = X_train.mean()
train_std = X_train.std()

X_train = (X_train - train_mean) / train_std
X_test = (X_test - train_mean) / train_std

print(X_train.shape)
X_train.describe()

(350, 4)


,Local_X(t-4),Local_X(t-3),Local_X(t-2),Local_X(t-1)
count,3.500000e+02,3.500000e+02,3.500000e+02,3.500000e+02
mean,6.394885e-16,1.116567e-15,7.765217e-16,-8.120488e-17
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.585158e+00,-1.591476e+00,-1.602768e+00,-1.611491e+00
25%,-1.048653e+00,-1.049085e+00,-1.046041e+00,-1.047701e+00
50%,3.973715e-01,3.926763e-01,3.894742e-01,3.853394e-01
75%,9.683812e-01,9.686760e-01,9.662557e-01,9.624222e-01
max,1.251361e+00,1.245572e+00,1.242927e+00,1.243391e+00


### Reshape data sets


In [16]:
X_train = X_train.values
X_test = X_test.values
# reshape into [# samples, # timesteps,# features]
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],1))
X_test = X_test.reshape((X_test.shape[0], X_train.shape[1],1))

In [20]:
X_train = torch.Tensor(X_train)
X_test = torch.Tensor(X_test)

In [23]:
X_train.shape

torch.Size([350, 4, 1])

## Prediction model

Define the LSTM class

In [18]:
class LSTM(nn.Module):
    # define each layer
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))
    
    # Define how network move forward
    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

In [19]:
model = LSTM()
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
print(model)

LSTM(
  (lstm): LSTM(1, 100)
  (linear): Linear(in_features=100, out_features=1, bias=True)
)


In [21]:
epochs = 10
for i in range(epochs):
    for seq, labels in zip(X_train,y_train):
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                        torch.zeros(1, 1, model.hidden_layer_size))

        y_pred = model(seq)

        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()

    if i%25 == 1:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

AttributeError: 'str' object has no attribute 'size'

In [20]:
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1],1)))
model.add(Dense(n_labels*n_future))
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

# For saving the best model during the whole training process.
#checkpointer = callbacks.ModelCheckpoint(filepath='BestModel.h5', monitor='val_loss', save_best_only=True)

#### Interrupt training if `val_loss` stops improving for over 10 epochs #######
stop_learn= tf.keras.callbacks.EarlyStopping(patience=10, monitor='val_loss')


# fit model
Monitor = model.fit(X_train, y_train, epochs=100, 
                    callbacks=[stop_learn],
                    validation_data=(X_test, y_test), verbose=1)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 704 samples, validate on 302 samples


InternalError: GPU sync failed

In [ ]:
hist = pd.DataFrame(Monitor.history)
hist['epoch'] = Monitor.epoch
fig, axes = plt.subplots(nrows=1, ncols=2,figsize=(10,4),dpi=150)
hist[['loss','val_loss']].plot(ax=axes[0])
hist[['mse','val_mse']].plot(ax=axes[1])
plt.show()
hist.tail()

## Evaluation 


In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
yhat = model.predict(X_test, verbose=1)
rms = [sqrt(mean_squared_error(y_test, yhat))]
print(yhat[:5])
rms

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(y_test.index,y_test, label = "true Local_X at t",marker = 'X', )
plt.scatter(y_test.index,yhat, label = "prediction Local_X at t",marker = '.')
plt.legend()
plt.show()

# Model 2

|Model name|Description| # Car|# vars|# targets|# const_vars| # steps | # futures
|---|:--|:-:|:-:|:-:|:-:|:-:|:-:|
Model 2|Add multi output time steps|1|1|1|0|4|**2**|

## Data preparation

We will use the same data set as model 1

In [ ]:
n_steps = 4
n_labels = 1
n_future = 2
n_features = 1

In [ ]:
X, y = series2seq1(simple_df.Local_X, n_in=n_steps, n_out=n_future, dropnan=True)
print(X.head(), X.shape)
print(y.head(), y.shape)

### Split the data set


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, 
                                                    test_size=0.3, random_state=42)
print(X_train.shape,X_test.shape, y_train.shape, y_test.shape)

In [ ]:
### Standardize the data
train_mean = X_train.mean()
train_std = X_train.std()

X_train = (X_train - train_mean) / train_std
X_test = (X_test - train_mean) / train_std

print(X_train.shape)
X_train.describe()

### Reshape data sets


In [ ]:
X_train = X_train.values
X_test = X_test.values
# reshape into [# samples, # timesteps,# features]
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],1))
X_test = X_test.reshape((X_test.shape[0], X_train.shape[1],1))

## Prediction model

In [ ]:
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1],1)))
model.add(Dense(n_labels*n_future))
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

# For saving the best model during the whole training process.
#checkpointer = callbacks.ModelCheckpoint(filepath='BestModel.h5', monitor='val_loss', save_best_only=True)

#### Interrupt training if `val_loss` stops improving for over 10 epochs #######
stop_learn= tf.keras.callbacks.EarlyStopping(patience=10, monitor='val_loss')


# fit model
Monitor = model.fit(X_train, y_train, epochs=100, 
                    callbacks=[stop_learn],
                    validation_data=(X_test, y_test), verbose=1)

In [ ]:
hist = pd.DataFrame(Monitor.history)
hist['epoch'] = Monitor.epoch
fig, axes = plt.subplots(nrows=1, ncols=2,figsize=(10,4),dpi=150)
hist[['loss','val_loss']].plot(ax=axes[0])
hist[['mse','val_mse']].plot(ax=axes[1])
plt.show()
hist.tail()

## Evaluation 


In [ ]:
yhat = model.predict(X_test, verbose=1)
rms += [sqrt(mean_squared_error(y_test, yhat))]
print(yhat[:5])
rms

In [ ]:
plt.figure(figsize=(15,6))
for i in range(n_future):
  plt.subplot(1,n_future,i+1)
  plt.scatter(y_test.index,y_test.iloc[:,i], label = "true Local_X at t+{}".format(i),marker = 'X', )
  plt.scatter(y_test.index,yhat[:,i], label = "prediction Local_X at t+{}".format(i),marker = '.')
  plt.legend()
plt.show()

# Model 3

|Model name|Description| # Car|# vars|# targets|# const_vars| # steps | # futures
|---|:--|:-:|:-:|:-:|:-:|:-:|:-:|
Model 3|Add multi input series|1|**2**|1|0|4|2|

## Data preparation

We will add one more feature v_Vel to the data set used in model 1 and 2

In [ ]:
n_steps = 4
n_future = 2
n_features = 2
feature_names = ['Local_X','v_Vel']
target_names = ['Local_X']
n_labels = len(target_names)
n_features = len(feature_names)

### series2seq2: Function that return sequence input and output| Version 2

**Arguments**:

- data: Sequence of observations as a list or NumPy array.
- n_in: Number of lag observations as input (X).
- n_out: Number of observations as output (y).
- **labels**: name of target variables
- dropnan: Boolean whether or not to drop rows with NaN values.
    
**Returns**:
- X: Feature Pandas DataFrame
- y: Label Pandas dataframe

  

In [ ]:
def series2seq2(data, n_in=1, n_out=1,labels=None, dropnan=True):

    dat = data.copy()
    if isinstance(dat,pd.Series):
      features = [dat.name]
      targets = [dat.name]
    else:
      features = dat.columns
      if labels == None:
        targets = dat.columns
      else:
        targets = labels
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(dat.shift(i))
        names += ['{}(t-{})'.format(j, i) for j in features ]
    # forecast sequence (t, t+1, ... t+n) for selected labels
    for i in range(0, n_out):
        cols.append(dat[targets].shift(-i))
        if i == 0:
            names += ['{}(t)'.format(j) for j in targets]
        else:
            names += ['{}(t+{})'.format(j, i) for j in targets]
    # put it all together
    agg = pd.concat(cols, axis=1)
    
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    X = agg.iloc[:,:len(features)*n_in]
    y = agg.iloc[:,len(features)*n_in:].copy()
    print(X.head(), X.shape)
    print(y.head(), y.shape)
    return X, y
  
# Test the function
X, y = series2seq2(df[feature_names], n_in=2, n_out=1,labels = target_names, dropnan=False)

In [ ]:
X, y = series2seq2(simple_df[feature_names], n_in=n_steps, n_out=n_future,labels = target_names, dropnan=True)

### Split the data set


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, 
                                                    test_size=0.3, random_state=42)
print(X_train.shape,X_test.shape, y_train.shape, y_test.shape)

In [ ]:
### Standardize the data
train_mean = X_train.mean()
train_std = X_train.std()

X_train = (X_train - train_mean) / train_std
X_test = (X_test - train_mean) / train_std

print(X_train.shape)
X_train.describe()

### Reshape data sets


In [ ]:
X_train = X_train.values
X_test = X_test.values
# reshape into [# samples, # timesteps,# features]
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],1))
X_test = X_test.reshape((X_test.shape[0], X_train.shape[1],1))

## Prediction model

In [ ]:
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1],1)))
model.add(Dense(n_labels*n_future))
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

# For saving the best model during the whole training process.
#checkpointer = callbacks.ModelCheckpoint(filepath='BestModel.h5', monitor='val_loss', save_best_only=True)

#### Interrupt training if `val_loss` stops improving for over 10 epochs #######
stop_learn= tf.keras.callbacks.EarlyStopping(patience=10, monitor='val_loss')


# fit model
Monitor = model.fit(X_train, y_train, epochs=100, 
                    callbacks=[stop_learn],
                    validation_data=(X_test, y_test), verbose=1)

In [ ]:
hist = pd.DataFrame(Monitor.history)
hist['epoch'] = Monitor.epoch
fig, axes = plt.subplots(nrows=1, ncols=2,figsize=(10,4),dpi=150)
hist[['loss','val_loss']].plot(ax=axes[0])
hist[['mse','val_mse']].plot(ax=axes[1])
plt.show()
hist.tail()

## Evaluation 


In [ ]:
yhat = model.predict(X_test, verbose=1)
rms += [sqrt(mean_squared_error(y_test, yhat))]
print(yhat[:5])
rms

In [ ]:
plt.figure(figsize=(15,6))
k = 1
for i in range(n_future):
  for j in range(1,n_labels+1):
    plt.subplot(j,n_future,k)
    plt.scatter(y_test.index,y_test.iloc[:,i], label = "true Local_X at t+{}".format(i),marker = 'X', )
    plt.scatter(y_test.index,yhat[:,i], label = "prediction Local_X at t+{}".format(i),marker = '.')
    plt.legend()
    k+=1
plt.show()

# Model 4

|Model name|Description| # Car|# vars|# targets|# const_vars| # steps | # futures
|---|:--|:-:|:-:|:-:|:-:|:-:|:-:|
Model 4|Add multi `output` series|1|2|**2**|0|4|2|

## Data preparation

We will use the same dataset of model 3

In [ ]:
n_steps = 4
n_future = 2
n_features = 2
feature_names = ['Local_X','v_Vel']
target_names = ['Local_X','v_Vel']
n_labels = len(target_names)
n_features = len(feature_names)

In [ ]:
X, y = series2seq2(simple_df[feature_names], n_in=n_steps, n_out=n_future,labels = target_names, dropnan=True)

### Split the data set


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, 
                                                    test_size=0.3, random_state=42)
print(X_train.shape,X_test.shape, y_train.shape, y_test.shape)

In [ ]:
### Standardize the data
train_mean = X_train.mean()
train_std = X_train.std()

X_train = (X_train - train_mean) / train_std
X_test = (X_test - train_mean) / train_std

print(X_train.shape)
X_train.describe()

### Reshape data sets


In [ ]:
X_train = X_train.values
X_test = X_test.values
# reshape into [# samples, # timesteps,# features]
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],1))
X_test = X_test.reshape((X_test.shape[0], X_train.shape[1],1))

## Prediction model

In [ ]:
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1],1)))
model.add(Dense(n_labels*n_future))
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

# For saving the best model during the whole training process.
#checkpointer = callbacks.ModelCheckpoint(filepath='BestModel.h5', monitor='val_loss', save_best_only=True)

#### Interrupt training if `val_loss` stops improving for over 10 epochs #######
stop_learn= tf.keras.callbacks.EarlyStopping(patience=10, monitor='val_loss')


# fit model
Monitor = model.fit(X_train, y_train, epochs=100, 
                    callbacks=[stop_learn],
                    validation_data=(X_test, y_test), verbose=1)

In [ ]:
hist = pd.DataFrame(Monitor.history)
hist['epoch'] = Monitor.epoch
fig, axes = plt.subplots(nrows=1, ncols=2,figsize=(10,4),dpi=150)
hist[['loss','val_loss']].plot(ax=axes[0])
hist[['mse','val_mse']].plot(ax=axes[1])
plt.show()
hist.tail()

## Evaluation 


In [ ]:
yhat = model.predict(X_test, verbose=1)
rms += [sqrt(mean_squared_error(y_test, yhat))]
print(yhat[:5])
rms

In [ ]:
plt.figure(figsize=(n_labels*5,n_future*5))
k = 1
for i in range(n_future):
  for j in range(1,n_labels+1):
    plt.subplot(n_labels,n_future,k)
    plt.scatter(y_test.index,y_test.iloc[:,i], label = "true {} at t+{}".format(target_names[j-1],i),marker = 'X', )
    plt.scatter(y_test.index,yhat[:,i], label = "prediction {} at t+{}".format(target_names[j-1],i),marker = '.')
    plt.legend()
    k+=1
plt.show()

# Model 5

|Model name|Description| # Car|# vars|# targets|# const_vars| # steps | # futures
|---|:--|:-:|:-:|:-:|:-:|:-:|:-:|
Model 5|Add multi constant input|1|2|2|**2**|4|2|

## Data preparation

We will add one two constant features `v_Length`, `v_Width` to the data set used in model 3 and 4

In [ ]:
simple_df[[ 'Local_X', 'v_Vel', 'v_Length', 'v_Width']].head()

In [ ]:
n_steps = 4
n_future = 2
n_features = 2
series_feature_names = ['Local_X','v_Vel']
target_names = ['Local_X','v_Vel']
n_labels = len(target_names)
n_features = len(feature_names)
const_feats = []

### series2seq3: Function that return sequence input and output| Version 3

**Arguments**:

- data: Sequence of observations as a list or NumPy array.
- n_in: Number of lag observations as input (X).
- n_out: Number of observations as output (y).
- **series_features**: names of series features
- labels: name of target variables
- dropnan: Boolean whether or not to drop rows with NaN values.
    
**Returns**:
- X: Feature Pandas DataFrame
- y: Label Pandas dataframe

  

In [ ]:
def series2seq3(data, n_in=1, n_out=1,labels=None,series_features=None, dropnan=True):

    dat = data.copy()
    # check if the data set is univariable
    if isinstance(dat,pd.Series):
      features = [dat.name]
      targets = [dat.name]
    else:
      features = dat.columns
      if labels == None:
        targets = dat.columns
      else:
        targets = labels
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    if series_features == None:
      for i in range(n_in, 0, -1):
        cols.append(dat.shift(i))
        names += ['{}(t-{})'.format(j, i) for j in features ]
    else:
      for i in range(n_in, 0, -1):
        cols.append(dat[series_features].shift(i))
        names += ['{}(t-{})'.format(j, i) for j in series_features]
    # forecast sequence (t, t+1, ... t+n) for selected labels
    #print(targets)
    for i in range(0, n_out):
        cols.append(dat[targets].shift(-i))
        if i == 0:
            names += ['{}(t)'.format(j) for j in targets]
        else:
            names += ['{}(t+{})'.format(j, i) for j in targets]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # concatenate with constant features
    
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    X = agg.iloc[:,:len(series_features)*n_in]
    X = pd.concat([X,dat.drop(columns=series_features)], axis=1).dropna()
    y = agg.iloc[:,len(series_features)*n_in:].copy()
    #print(X.head(), X.shape)
    #print(y.head(), y.shape)
    return X, y
  
# Test the function
X, y = series2seq3(simple_df[[ 'Local_X', 'v_Vel', 'v_Length', 'v_Width']], n_in=2, n_out=1,labels = target_names,series_features=series_feature_names, dropnan=True)

In [ ]:
X, y = series2seq3(simple_df[[ 'Local_X', 'v_Vel', 'v_Length', 'v_Width']], 
                   n_in=n_steps, n_out=n_future,
                   labels = target_names,
                   series_features=series_feature_names, 
                   dropnan=True)

### Split the data set


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, 
                                                    test_size=0.3, random_state=42)
print(X_train.shape,X_test.shape, y_train.shape, y_test.shape)

In [ ]:
X_train.describe()

### Reshape data sets


In [ ]:
X_train = X_train.values
X_test = X_test.values
# reshape into [# samples, # timesteps,# features]
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],1))
X_test = X_test.reshape((X_test.shape[0], X_train.shape[1],1))

## Prediction model

In [ ]:
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1],1)))
model.add(Dense(n_labels*n_future))
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

# For saving the best model during the whole training process.
#checkpointer = callbacks.ModelCheckpoint(filepath='BestModel.h5', monitor='val_loss', save_best_only=True)

#### Interrupt training if `val_loss` stops improving for over 10 epochs #######
stop_learn= tf.keras.callbacks.EarlyStopping(patience=10, monitor='val_loss')


# fit model
Monitor = model.fit(X_train, y_train, epochs=100, 
                    callbacks=[stop_learn],
                    validation_data=(X_test, y_test), verbose=1)

In [ ]:
hist = pd.DataFrame(Monitor.history)
hist['epoch'] = Monitor.epoch
fig, axes = plt.subplots(nrows=1, ncols=2,figsize=(10,4),dpi=150)
hist[['loss','val_loss']].plot(ax=axes[0])
hist[['mse','val_mse']].plot(ax=axes[1])
plt.show()
hist.tail()

## Evaluation 


In [ ]:
yhat = model.predict(X_test, verbose=1)
rms += [sqrt(mean_squared_error(y_test, yhat))]
print(yhat[:5])
rms

In [ ]:
plt.figure(figsize=(n_labels*5,n_future*5))
k = 1
for i in range(n_future):
  for j in range(1,n_labels+1):
    plt.subplot(n_labels,n_future,k)
    plt.scatter(y_test.index,y_test.iloc[:,i], label = "true {} at t+{}".format(target_names[j-1],i),marker = 'X', )
    plt.scatter(y_test.index,yhat[:,i], label = "prediction {} at t+{}".format(target_names[j-1],i),marker = '.')
    plt.legend()
    k+=1
plt.show()

# Model 6

|Model name|Description| # Car|# vars|# targets|# const_vars| # steps | # futures
|---|:--|:-:|:-:|:-:|:-:|:-:|:-:|
Model 6|Add more objects|**4**|2|2|2|4|2|

## Data preparation

We will add 3 more objects to the dataset used in previous models

In [ ]:
np.random.seed(23)
veh_list = np.random.choice(df.Vehicle_ID.unique(),3)
veh_list = np.append(veh_list,2582)
veh_list

In [ ]:
kept_cols = [ 'Vehicle_ID', 'Frame_ID','Local_X', 'v_Vel', 'v_Length', 'v_Width']
simple_df = df[df.Vehicle_ID.isin(veh_list)].copy()[kept_cols]
simple_df.info()
simple_df.head()

In [ ]:
n_steps = 4
n_future = 2
n_features = 2
series_feature_names = ['Local_X','v_Vel']
target_names = ['Local_X','v_Vel']
n_labels = len(target_names)
n_features = len(feature_names)

### `treatment_cars` Function to prepare the data set for each car

In [ ]:
def treatment_cars(data, n_in=1, n_out=1,labels=None,series_features=None):
  veh_ids = data.Vehicle_ID.unique()
  dat_X, dat_y = pd.DataFrame(),pd.DataFrame()

  for id in veh_ids:
    dat = data[data.Vehicle_ID==id].copy()
    X, y = series2seq3(dat.drop(columns=['Frame_ID']), n_in=n_in, n_out=n_out,labels = labels,series_features=series_features, dropnan=True)
    dat_X = pd.concat([dat_X,X],ignore_index=True)
    dat_y = pd.concat([dat_y,y],ignore_index=True)
  print(dat_X.shape)
  print(dat_y.shape)
  return dat_X ,dat_y
treatment_cars(simple_df,n_in=2, n_out=1, labels = target_names,series_features=series_feature_names,)

In [ ]:
X, y = treatment_cars(simple_df, 
                   n_in=n_steps, n_out=n_future,
                   labels = target_names,
                   series_features=series_feature_names)

In [ ]:
X.Vehicle_ID.unique()

### Split the data set


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, 
                                                    test_size=0.3, random_state=42)
print(X_train.shape,X_test.shape, y_train.shape, y_test.shape)

In [ ]:
X_train.describe()

In [ ]:
### Standardize the data
train_mean = X_train.mean()
train_std = X_train.std()

X_train = (X_train - train_mean) / train_std
X_test = (X_test - train_mean) / train_std

print(X_train.shape)
X_train.describe()

### Reshape data sets


In [ ]:
X_train = X_train.values
X_test = X_test.values
# reshape into [# samples, # timesteps,# features]
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],1))
X_test = X_test.reshape((X_test.shape[0], X_train.shape[1],1))

## Prediction model

In [ ]:
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1],1)))
model.add(Dense(n_labels*n_future))
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

# For saving the best model during the whole training process.
#checkpointer = callbacks.ModelCheckpoint(filepath='BestModel.h5', monitor='val_loss', save_best_only=True)

#### Interrupt training if `val_loss` stops improving for over 10 epochs #######
stop_learn= tf.keras.callbacks.EarlyStopping(patience=10, monitor='val_loss')


# fit model
Monitor = model.fit(X_train, y_train, epochs=100, 
                    callbacks=[stop_learn],
                    validation_data=(X_test, y_test), verbose=1)

In [ ]:
hist = pd.DataFrame(Monitor.history)
hist['epoch'] = Monitor.epoch
fig, axes = plt.subplots(nrows=1, ncols=2,figsize=(10,4),dpi=150)
hist[['loss','val_loss']].plot(ax=axes[0])
hist[['mse','val_mse']].plot(ax=axes[1])
plt.show()
hist.tail()

## Evaluation 


In [ ]:
yhat = model.predict(X_test, verbose=1)
rms += [sqrt(mean_squared_error(y_test, yhat))]
print(yhat[:5])
rms

In [ ]:
plt.figure(figsize=(n_labels*7,n_future*5))
k = 0
for i in range(n_future):
  for j in range(n_labels):
    plt.subplot(n_labels,n_future,k+1)
    plt.scatter(y_test.index,y_test.iloc[:,k], label = "true {} at t+{}".format(target_names[j],i),marker = 'X', )
    plt.scatter(y_test.index,yhat[:,k], label = "prediction {} at t+{}".format(target_names[j],i),marker = '.')
    plt.legend()
    k+=1
plt.show()

# 1000 cars



## Data preparation

We will add 3 more objects to the dataset used in previous models

In [ ]:
np.random.seed(23)
veh_list = np.random.choice(df.Vehicle_ID.unique(),1000)

In [ ]:
kept_cols = [ 'Vehicle_ID', 'Frame_ID','Local_X', 'v_Vel', 'v_Length', 'v_Width']
simple_df = df[df.Vehicle_ID.isin(veh_list)].copy()[kept_cols]
simple_df.info()
simple_df.head()

In [ ]:
n_steps = 4
n_future = 2
n_features = 2
series_feature_names = ['Local_X','v_Vel']
target_names = ['Local_X','v_Vel']
n_labels = len(target_names)
n_features = len(feature_names)

### `treatment_cars` Function to prepare the data set for each car

In [ ]:
def treatment_cars(data, n_in=1, n_out=1,labels=None,series_features=None):
  veh_ids = data.Vehicle_ID.unique()
  dat_X, dat_y = pd.DataFrame(),pd.DataFrame()

  for id in veh_ids:
    dat = data[data.Vehicle_ID==id].copy()
    X, y = series2seq3(dat.drop(columns=['Frame_ID']), n_in=n_in, n_out=n_out,labels = labels,series_features=series_features, dropnan=True)
    dat_X = pd.concat([dat_X,X],ignore_index=True)
    dat_y = pd.concat([dat_y,y],ignore_index=True)
  print(dat_X.shape)
  print(dat_y.shape)
  return dat_X ,dat_y
treatment_cars(simple_df,n_in=2, n_out=1, labels = target_names,series_features=series_feature_names,)

In [ ]:
X, y = treatment_cars(simple_df, 
                   n_in=n_steps, n_out=n_future,
                   labels = target_names,
                   series_features=series_feature_names)

In [ ]:
len(X.Vehicle_ID.unique())

### Split the data set


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, 
                                                    test_size=0.3, random_state=42)
print(X_train.shape,X_test.shape, y_train.shape, y_test.shape)

In [ ]:
X_train.describe()

In [ ]:
### Standardize the data
train_mean = X_train.mean()
train_std = X_train.std()

X_train = (X_train - train_mean) / train_std
X_test = (X_test - train_mean) / train_std

print(X_train.shape)
X_train.describe()

### Reshape data sets


In [ ]:
X_train = X_train.values
X_test = X_test.values
# reshape into [# samples, # timesteps,# features]
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],1))
X_test = X_test.reshape((X_test.shape[0], X_train.shape[1],1))

## Prediction model

In [ ]:
import time
time.time()

In [ ]:
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1],1)))
model.add(Dense(n_labels*n_future))
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

# For saving the best model during the whole training process.
#checkpointer = callbacks.ModelCheckpoint(filepath='BestModel.h5', monitor='val_loss', save_best_only=True)

#### Interrupt training if `val_loss` stops improving for over 10 epochs #######
stop_learn= tf.keras.callbacks.EarlyStopping(patience=10, monitor='val_loss')

start = time.time()
# fit model
Monitor = model.fit(X_train, y_train, epochs=100, 
                    callbacks=[stop_learn],
                    validation_data=(X_test, y_test), verbose=1)
end = time.time()
'Training time was {} sec'.format(end-start)

In [ ]:
hist = pd.DataFrame(Monitor.history)
hist['epoch'] = Monitor.epoch
fig, axes = plt.subplots(nrows=1, ncols=2,figsize=(10,4),dpi=150)
hist[['loss','val_loss']].plot(ax=axes[0])
hist[['mse','val_mse']].plot(ax=axes[1])
plt.show()
hist.tail()

## Evaluation 


In [ ]:
yhat = model.predict(X_test, verbose=1)
rms += [sqrt(mean_squared_error(y_test, yhat))]
print(yhat[:5])
rms

In [ ]:
plt.figure(figsize=(n_labels*5,n_future*5))
k = 1
for i in range(n_future):
  for j in range(1,n_labels+1):
    plt.subplot(n_labels,n_future,k)
    plt.scatter(y_test.index,y_test.iloc[:,i], label = "true {} at t+{}".format(target_names[j-1],i),marker = 'X', )
    plt.scatter(y_test.index,yhat[:,i], label = "prediction {} at t+{}".format(target_names[j-1],i),marker = '.')
    plt.legend()
    k+=1
plt.show()

# Conclusion

Overview on the performance of all simple models

|Model name|Description| # Car|# vars|# targets|# const_vars| # steps | # futures|RMS|Computing time|
|---|:--|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
Model 1|Baseline model with multi input time steps|1|1|1|0|**4**|1|0.307|0.1sec| 
Model 2|Add multi output time steps|1|1|1|0|4|**2**|0.315|0.1|
Model 3|Add multi input series|1|**2**|1|0|4|2|0.412|0.1|
Model 4|Add multi `output` series|1|2|**2**|0|4|2| 0.831|0.1|
Model 5|Add multi constant input|1|2|2|**2**|4|2|0.586|0.1|
Model 6|Add more objects|**4**|2|2|2|4|2|0.752|0.1
Model 6.1|Add much more objects|**1000**|2|2|2|4|2|0.385|2200sec (37min)

#END
